In [33]:
import pandas as pd
data = pd.read_csv('features/features.csv', index_col='match_id')
features_test = pd.read_csv('features_test.csv', index_col='match_id')


## Пропущенные признаки

In [34]:
data.describe().T['count'][data.describe().T['count'] < len(data)].sort_values().apply(lambda x: (len(data) - x) / len(data))

first_blood_player2            0.452402
radiant_flying_courier_time    0.282619
dire_flying_courier_time       0.268415
first_blood_time               0.201100
first_blood_team               0.201100
first_blood_player1            0.201100
dire_bottle_time               0.166029
radiant_bottle_time            0.161380
radiant_first_ward_time        0.018883
dire_first_ward_time           0.018780
radiant_courier_time           0.007117
dire_courier_time              0.006953
Name: count, dtype: float64

### Данные за 5 минут игры
#### справа вероятность того, что событие не произойдет за первые 5 минут игры

* **first_blood_player2** - второй игрок, причастный к событию. Отсутвует, потому что за 5 минут никого не убили /убил только один игрок.
* **radiant_flying_courier_time** - время приобретения предмета "flying_courier". Отсутствует, потому что за 5 минут не был куплен.
* **dire_flying_courier_time** - аналогично со стороны другой команды
* **first_blood_time** - игровое время первой крови. Отсутсвует, потому что за 5 минут никого не убили
* **first_blood_team** - аналогичная причина
* **first_blood_player1** - аналогичная причина
* **dire_bottle_time** - время первого приобретения командой предмета "bottle". Отсутствует, потому что никто не купил за  5 минут.
* **radiant_bottle_time** - аналогичная причина.
* **radiant_first_ward_time** -  время установки командой первого "наблюдателя". Отсутствует, потому что никто не установил за  5 минут.
* **dire_first_ward_time** - аналогичная причина для другой команды
* **radiant_courier_time** - время приобретения предмета "courier".Отсутствует, потому что за 5 минут не был куплен.
* **dire_courier_time** - аналогичная причина для другой команды.

In [35]:

data.fillna(0,inplace = True)
data.drop(['duration', 
         'tower_status_radiant', 
         'tower_status_dire', 
         'barracks_status_radiant', 
         'barracks_status_dire'], axis=1, inplace=True)


In [36]:
X_train = data
#y_train = data['radiant_win'].to_frame()
#y_train = data['radiant_win'].to_numpy

y_train = data['radiant_win'].values
X_train.drop(['radiant_win'],axis = 1,inplace = True)


**radiant_win** - целевая переменная

## Градиентный бустинг

In [6]:
from sklearn.model_selection import KFold, cross_val_score
from datetime import datetime
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np

cv = KFold(n_splits = 5, shuffle=True)
scores = list()
for k in (10,20,30,50,100):
    start_time = datetime.now()
    model = GradientBoostingClassifier(n_estimators=k)
    scores.append(cross_val_score(model, X_train, y_train, cv=cv, scoring='roc_auc')) 
    print ('#' + str(k) + ' Time:  ' + str(datetime.now() - start_time))
    print ('SCORES:  ' + str(scores[-1]) + '    mean:   ' + str(np.mean(scores[-1])))
    print()

#10 Time:  0:00:41.126477
SCORES:  [0.66250835 0.66515185 0.65618036 0.66989025 0.67549534]    mean:   0.6658452314180121

#20 Time:  0:01:19.380765
SCORES:  [0.67432171 0.68895313 0.68274943 0.6809305  0.68524744]    mean:   0.6824404426871318

#30 Time:  0:01:59.220209
SCORES:  [0.68703857 0.69300469 0.68887393 0.69159849 0.68852907]    mean:   0.6898089476941627

#50 Time:  0:03:17.936684
SCORES:  [0.69987485 0.69585647 0.68978932 0.70056541 0.69801329]    mean:   0.6968198689979208

#100 Time:  0:06:34.240674
SCORES:  [0.69684333 0.7061727  0.71073256 0.70831198 0.70992228]    mean:   0.7063965710100316



 #### Время выполнения Кросс-валидации для разного количества деревьев с соответствующими значениями метрики AUC-ROC


Имеет смысл использовать больше 30 деревьев, так как с увелечением количества деревьев увеличиваются значения метрики AUC-ROC
Что делать, чтобы ускорить его обучение при увеличении количества деревьев:
*Использовать для обучения и кросс-валидации не всю выборку, а некоторое ее подмножество — например, половину объектов. *Подмножество лучше всего брать случайным, а не формировать его из первых m объектов.
*Попробовать упростить модель — например, уменьшить глубину деревьев в градиентом бустинге (max_depth).

## Логистическая регрессия


In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

X_train = StandardScaler().fit_transform(X_train)

scores = list()

for c in [10 ** i for i in range(-3, 5)]:
    start_time = datetime.now()
    
    model = LogisticRegression(C = c)
    scores.append(cross_val_score(model, X_train, y_train, cv=cv, scoring='roc_auc'))
    
    print ('#' + str(c) + ' Time:  ' + str(datetime.now() - start_time))
    print ('SCORES:  ' + str(scores[-1]) + '    mean:   ' + str(np.mean(scores[-1])))
    print()

#0.001 Time:  0:00:02.073783
SCORES:  [0.71186626 0.71410819 0.71357438 0.72176492 0.72011222]    mean:   0.7162851929384727

#0.01 Time:  0:00:02.713743
SCORES:  [0.72328302 0.71348524 0.71574974 0.71529435 0.71366313]    mean:   0.7162950961683041

#0.1 Time:  0:00:03.025908
SCORES:  [0.71816765 0.7126706  0.71585986 0.71820783 0.71619713]    mean:   0.7162206159282769

#1 Time:  0:00:02.900245
SCORES:  [0.71675218 0.71390773 0.71505026 0.7217247  0.7152278 ]    mean:   0.7165325335142194

#10 Time:  0:00:02.825443
SCORES:  [0.71922478 0.7169698  0.71513777 0.71093017 0.7186686 ]    mean:   0.7161862231434173

#100 Time:  0:00:02.909258
SCORES:  [0.71929308 0.71275094 0.71608428 0.71538901 0.71924203]    mean:   0.7165518679271665

#1000 Time:  0:00:02.863349
SCORES:  [0.71444112 0.71937985 0.71797177 0.71685847 0.71429735]    mean:   0.7165897109520365

#10000 Time:  0:00:02.960084
SCORES:  [0.7122857  0.71767674 0.71424461 0.7193226  0.71799037]    mean:   0.7163040032090848



#### Время выполнения Кросс-Валидации для разного параметра регуляризации С с соответствующими значениями метрики AUC-ROC


Лучшее значение метрики AUC-ROC = 0.716 при C = 1. Это выше чем при бустинге со 100 деревьявми.

In [31]:
data.drop([
    'lobby_type',
    'r1_hero',
    'r2_hero',
    'r3_hero',
    'r4_hero',
    'r5_hero',
    'd1_hero',
    'd2_hero',
    'd3_hero',
    'd4_hero',
    'd5_hero',
],axis = 1, inplace = True)

X_train = data
X_train = StandardScaler().fit_transform(X_train)

In [9]:

scores = list()

for c in [10 ** i for i in range(-3, 5)]:
    start_time = datetime.now()
    
    model = LogisticRegression(C = c)
    scores.append(cross_val_score(model, X_train, y_train, cv=cv, scoring='roc_auc'))
    
    print ('#' + str(c) + ' Time:  ' + str(datetime.now() - start_time))
    print ('SCORES:  ' + str(scores[-1]) + '    mean:   ' + str(np.mean(scores[-1])))
    print()

#0.001 Time:  0:00:01.827617
SCORES:  [0.71645095 0.71440459 0.71682396 0.71771616 0.71564768]    mean:   0.7162086657171703

#0.01 Time:  0:00:02.661887
SCORES:  [0.70912653 0.71675918 0.71731267 0.71714265 0.72154518]    mean:   0.7163772417330774

#0.1 Time:  0:00:02.625977
SCORES:  [0.71896721 0.71754684 0.7221589  0.71268791 0.71200606]    mean:   0.7166733832005938

#1 Time:  0:00:02.617007
SCORES:  [0.71650863 0.71565283 0.71390478 0.72040674 0.71553033]    mean:   0.7164006603859998

#10 Time:  0:00:02.666868
SCORES:  [0.7197465  0.71984904 0.71698703 0.71058957 0.71544023]    mean:   0.7165224736844957

#100 Time:  0:00:02.669912
SCORES:  [0.71637253 0.71694294 0.71493268 0.71664862 0.71685134]    mean:   0.716349623162843

#1000 Time:  0:00:02.688810
SCORES:  [0.71730523 0.71513047 0.71645226 0.71612433 0.71832224]    mean:   0.7166669039740172

#10000 Time:  0:00:02.617597
SCORES:  [0.71940238 0.71208107 0.71940506 0.7113497  0.7198608 ]    mean:   0.7164197991848217



Лучшее значение метрики AUC-ROC = 0.7164 при C = 1000 и это лучше, чем было без удаления категориальных признаков, однако значение регуляризации тоже несущественно поменялось, потому что мы не ставили random_state.

In [37]:
print(str(len(np.unique(data[['r' + str(i) + '_hero' for i in range(1,6)]]))) + ' уникальных героя')

108 уникальных героя


### Bag of word

In [38]:
import pandas as pd
X_pick = np.zeros((data.shape[0], 112))

for i, match_id in enumerate(data.index):
    for p in range(5):
        X_pick[i, data.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1
        
data.drop([
    'lobby_type',
    'r1_hero',
    'r2_hero',
    'r3_hero',
    'r4_hero',
    'r5_hero',
    'd1_hero',
    'd2_hero',
    'd3_hero',
    'd4_hero',
    'd5_hero',
],axis = 1, inplace = True)


In [39]:
X_train = np.hstack((data.values, X_pick))
X_train = StandardScaler().fit_transform(X_train)

array([[-2.54436416e+00,  1.40080818e+00,  1.52597175e+00, ...,
        -4.83865990e-03,  0.00000000e+00,  8.46061344e-04],
       [-2.54045236e+00,  5.01313538e-01, -8.01392943e-02, ...,
        -4.83865990e-03,  0.00000000e+00,  8.46061344e-04],
       [-2.53923104e+00,  5.01313538e-01,  1.51070097e-01, ...,
        -4.83865990e-03,  0.00000000e+00,  8.46061344e-04],
       ...,
       [ 1.09874571e+00,  5.01313538e-01,  2.92266672e-01, ...,
        -4.83865990e-03,  0.00000000e+00,  1.64609695e+00],
       [ 1.09895204e+00, -3.98181106e-01, -1.73682025e-01, ...,
        -4.83865990e-03,  0.00000000e+00,  8.46061344e-04],
       [ 1.10264790e+00, -3.98181106e-01, -3.18408515e-01, ...,
        -4.83865990e-03,  0.00000000e+00, -1.64440483e+00]])

In [40]:
scores = list()

for c in [10 ** i for i in range(-3, 5)]:
    start_time = datetime.now()
    
    model = LogisticRegression(C = c)
    scores.append(cross_val_score(model, X_train, y_train, cv=cv, scoring='roc_auc'))
    
    print ('#' + str(c) + ' Time:  ' + str(datetime.now() - start_time))
    print ('SCORES:  ' + str(scores[-1]) + '    mean:   ' + str(np.mean(scores[-1])))
    print()

#0.001 Time:  0:00:03.790863
SCORES:  [0.75190132 0.74459114 0.75545409 0.75135103 0.75501958]    mean:   0.75166343091473

#0.01 Time:  0:00:05.659863
SCORES:  [0.75292304 0.75308733 0.75533016 0.74740161 0.75118894]    mean:   0.7519862156189381

#0.1 Time:  0:00:06.148560
SCORES:  [0.75226126 0.75052564 0.74944457 0.75584924 0.75070509]    mean:   0.7517571593135237

#1 Time:  0:00:06.308131
SCORES:  [0.74612417 0.74919029 0.75585474 0.75025009 0.75738259]    mean:   0.7517603763946967

#10 Time:  0:00:06.271230
SCORES:  [0.75595914 0.74969909 0.75079595 0.75660447 0.74663206]    mean:   0.7519381438621213

#100 Time:  0:00:06.129607
SCORES:  [0.75326134 0.75294715 0.75518061 0.74912212 0.74911968]    mean:   0.7519261815511508

#1000 Time:  0:00:06.096697
SCORES:  [0.75278246 0.75160928 0.75283569 0.74964835 0.75141057]    mean:   0.7516572711560551

#10000 Time:  0:00:06.133596
SCORES:  [0.75687945 0.75049437 0.74888335 0.74886769 0.75387528]    mean:   0.7518000291659876




При добавления "мешка слов" по героям качество заметно улучшается. Лучшее значение метрики AUC-ROC = 0.75198, при C = 0.01. Объясняется тем, что информация какой герой использовался в матче дает полезную информацию. 

### Submit

In [43]:
data = pd.read_csv('data/features.csv',index_col='match_id')
data.drop(['duration', 
         'tower_status_radiant', 
         'tower_status_dire', 
         'barracks_status_radiant', 
         'barracks_status_dire',
         'radiant_win'
        ], axis=1, inplace=True)

data.fillna(0,inplace = True)

X_pick = np.zeros((data.shape[0], 112))

for i, match_id in enumerate(data.index):
    for p in range(5):
        X_pick[i, data.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1
        
data.drop([
    'lobby_type',
    'r1_hero',
    'r2_hero',
    'r3_hero',
    'r4_hero',
    'r5_hero',
    'd1_hero',
    'd2_hero',
    'd3_hero',
    'd4_hero',
    'd5_hero',
],axis = 1, inplace = True)


X_train = np.hstack((data.values, X_pick))
X_train = StandardScaler().fit_transform(X_train)
###


data = pd.read_csv('data/features_test.csv',index_col='match_id')
data.fillna(0,inplace = True)


X_pick = np.zeros((data.shape[0], 112))

for i, match_id in enumerate(data.index):
    for p in range(5):
        X_pick[i, data.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1
        
data.drop([
    'lobby_type',
    'r1_hero',
    'r2_hero',
    'r3_hero',
    'r4_hero',
    'r5_hero',
    'd1_hero',
    'd2_hero',
    'd3_hero',
    'd4_hero',
    'd5_hero',
],axis = 1, inplace = True)

X_test = np.hstack((data.values, X_pick))
X_test = StandardScaler().fit_transform(X_test)

In [44]:
model = LogisticRegression(C=0.01)
model.fit(X_train, y_train)

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [46]:
submit = model.predict_proba(X_test)
submit = [submit[i][1] for i in range(0,len(submit))]

In [47]:

print('Max proba: ' + str(max(submit)))
print('Min proba: ' + str(min(submit)))

Max proba: 0.9963305890721924
Min proba: 0.008721474723112774


In [48]:
pd.DataFrame(data = submit, index = data.index, columns=['radiant_win']).to_csv('kaggle_submit.csv')

Результат с kaggle - 0.75532